In [ ]:
!pip install -q numpy pillow torch torchvision requests scipy matplotlib opencv-python
!pip install -q git+https://github.com/geetu040/transformers.git@depth-pro-projects#egg=transformers
!pip install --upgrade pip --quiet

# MODEL


In [ ]:
import os
import time
import torch
import cv2
import requests
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from huggingface_hub import hf_hub_download
from torchvision import models
from torchvision.transforms import transforms
from scipy.ndimage import gaussian_filter
from scipy.spatial.distance import cdist
from scipy.io import loadmat


In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")


In [ ]:
manual_image_processor = None
try:
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    input_size_common = (1536, 1536) # Common size for models if needed, DepthPro expects this.
    manual_image_processor = transforms.Compose([
        transforms.Resize(input_size_common),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])
    print(f"Manual image preprocessor initialized with input size: {input_size_common}.")
except Exception as e:
    print(f"CRITICAL ERROR: Failed to initialize image processor. Please check torchvision installation: {e}")
    raise RuntimeError(f"Failed to initialize image processor: {e}")


# Behaviour



In [ ]:
try:
    from transformers import DepthProConfig, DepthProForDepthEstimation
except ImportError as e:
    print(f"Error importing core DepthPro model classes. Ensure 'git+https://github.com/geetu040/transformers.git@depth-pro-projects#egg=transformers' was installed correctly.")
    raise e

In [ ]:
csrnet_model = None
depthpro_model = None
CSRNET_MODEL_WEIGHTS_PATH = '/kaggle/input/weightfile/pytorch/default/1/model_weights_CSRNetConvNeXT12.pth'
DEPTHPRO_MODEL_REPO_ID = "geetu040/DepthPro_Segmentation_Human"
DEPTHPRO_MODEL_WEIGHTS_FILENAME = "model_weights.pth"

In [ ]:
try:
    # Initialize and Load CSRNet Model
    if csrnet_model is None:
        print(f"Initializing and loading CSRNet model from: {CSRNET_MODEL_WEIGHTS_PATH}")
        class CSRNet(nn.Module):
            def __init__(self):
                super().__init__()
                convNext = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.IMAGENET1K_V1)
                self.frontend = nn.Sequential(
                    convNext.features[0:4],
                    nn.Conv2d(192, 512, kernel_size=1)
                )
                self.backend = nn.Sequential(
                    nn.Conv2d(512, 512, kernel_size=3, padding=2, dilation=2),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(512, 512, kernel_size=3, padding=2, dilation=2),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(512, 512, kernel_size=3, padding=2, dilation=2),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(512, 256, kernel_size=3, padding=2, dilation=2),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(256, 128, kernel_size=3, padding=2, dilation=2),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(128, 64, kernel_size=3, padding=2, dilation=2),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(64, 1, kernel_size=1)
                )
            def forward(self, x):
                x = self.frontend(x)
                x = self.backend(x)
                return x

        csrnet_model = CSRNet()
        csrnet_model = csrnet_model.to(DEVICE)
        csrnet_model.load_state_dict(torch.load(CSRNET_MODEL_WEIGHTS_PATH, map_location=torch.device('cpu')))
        csrnet_model.eval()
        print(f"CSRNet model loaded successfully on {DEVICE}!")

    # Initialize and Load DepthPro Model
    if depthpro_model is None: # Only load if not already loaded
        print(f"Initializing and loading DepthPro model: {DEPTHPRO_MODEL_REPO_ID}")

        config = DepthProConfig(use_fov_model=False)
        depthpro_model = DepthProForDepthEstimation(config)

        features = config.fusion_hidden_size
        semantic_classifier_dropout = 0.1
        num_labels = 1
        depthpro_model.head.head = nn.Sequential(
            nn.Conv2d(features, features, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(features),
            nn.ReLU(),
            nn.Dropout(semantic_classifier_dropout),
            nn.Conv2d(features, features, kernel_size=1),
            nn.ConvTranspose2d(features, num_labels, kernel_size=2, stride=2, padding=0, bias=True),
        )

        weights_path = hf_hub_download(repo_id=DEPTHPRO_MODEL_REPO_ID, filename=DEPTHPRO_MODEL_WEIGHTS_FILENAME)
        depthpro_model.load_state_dict(torch.load(weights_path, map_location=torch.device('cpu'), weights_only=True))

        depthpro_model = depthpro_model.to(DEVICE)
        depthpro_model.eval()
        print(f"DepthPro model loaded successfully on {DEVICE}!")

except Exception as e:
    print(f"CRITICAL ERROR during model loading: {e}")
    raise RuntimeError(f"Failed to load one or more models: {e}")


In [ ]:
def show_mask(mask, ax, color=None):
    if color is None:
        color = np.array([30/255, 144/255, 255/255, 0.6]) # Default light blue
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(vis, lines, 0, (0, 255, 0)) # Green lines for flow
    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis
def flow_to_hsv(flow):
    # This function converts the flow field into an HSV image
    # Hue encodes direction, saturation encodes magnitude, Value is max
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)
    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = cv2.normalize(v, None, 0, 255, cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return rgb
def extract_frame(video_path, frame_number):
    """Extracts a specific frame (0-indexed) from a video."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return None
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    cap.release()
    return frame if ret else None

In [ ]:
def get_human_mask_and_density(image_bgr, upsampled_density_map, density_threshold_for_refinement=0.1):
    """
    Performs DepthPro inference for human segmentation and refines the mask using
    a predicted density map (e.g., from CSRNet).
    """
    image_pil = Image.fromarray(cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB))
    W, H = image_pil.size

    # --- DepthPro Inference ---
    input_tensor_depthpro = manual_image_processor(image_pil).unsqueeze(0).to(DEVICE)
    inputs_depthpro_dict = {'pixel_values': input_tensor_depthpro}

    with torch.no_grad():
        model_output_depthpro = depthpro_model(**inputs_depthpro_dict, return_dict=True)
        output_data_depthpro = model_output_depthpro.predicted_depth

    if output_data_depthpro.dim() == 3 and output_data_depthpro.shape[0] == 1:
        output_data_depthpro = output_data_depthpro.unsqueeze(1)

    # Upsample DepthPro output to original image size
    output_upsampled_depthpro = F.interpolate(
        output_data_depthpro,
        size=(H, W), # Use original H, W
        mode='bilinear',
        align_corners=False
    ).squeeze()

    output_sigmoid_depthpro = output_upsampled_depthpro.sigmoid()
    human_mask_from_model_depthpro = (output_sigmoid_depthpro > 0.2).float() # Using 0.2 threshold

    # --- Refine mask using the provided upsampled_density_map ---
    if not isinstance(upsampled_density_map, np.ndarray):
        print(f"Warning: upsampled_density_map is not a NumPy array. Type: {type(upsampled_density_map)}. Using zeros for refinement.")
        upsampled_density_map = np.zeros((H, W), dtype=np.float32) # Fallback

    density_mask_tensor = (torch.from_numpy(upsampled_density_map).to(DEVICE) > density_threshold_for_refinement).float()
    final_human_mask_tensor = torch.max(human_mask_from_model_depthpro, density_mask_tensor)

    human_mask_np = final_human_mask_tensor.cpu().numpy().astype(np.uint8) * 255
    
    return human_mask_np, upsampled_density_map

In [ ]:
MIN_DENSITY_FOR_METRIC = 1e-5 # Minimum density value for a pixel to be considered part of the crowd for metric calculation
MIN_FLOW_MAGNITUDE_FOR_WEIGHTING = 1e-5 

In [ ]:
def calculate_density_weighted_flow_metrics(masked_flow, density_map):
    """
    Calculates density-weighted average speed, dominant direction, and average flow components (dx, dy).
    Returns: avg_speed, dom_angle_degrees, avg_dx, avg_dy
    """
    total_weighted_dx = 0.0
    total_weighted_dy = 0.0
    total_density_sum = 0.0

    crowd_pixel_indices = np.argwhere(density_map > MIN_DENSITY_FOR_METRIC)

    if len(crowd_pixel_indices) == 0:
        return 0.0, 0.0, 0.0, 0.0

    for y, x in crowd_pixel_indices:
        dx, dy = masked_flow[y, x]
        density_value = density_map[y, x]

        if np.abs(dx) > MIN_FLOW_MAGNITUDE_FOR_WEIGHTING or np.abs(dy) > MIN_FLOW_MAGNITUDE_FOR_WEIGHTING:
            total_weighted_dx += dx * density_value
            total_weighted_dy += dy * density_value
            total_density_sum += density_value

    if total_density_sum == 0:
        return 0.0, 0.0, 0.0, 0.0

    avg_dx = total_weighted_dx / total_density_sum
    avg_dy = total_weighted_dy / total_density_sum

    average_speed = np.sqrt(avg_dx**2 + avg_dy**2)
    dominant_angle_radians = np.arctan2(avg_dy, avg_dx)
    dominant_angle_degrees = np.degrees(dominant_angle_radians)

    if dominant_angle_degrees < 0:
        dominant_angle_degrees += 360

    return average_speed, dominant_angle_degrees, avg_dx, avg_dy



def calculate_flow_magnitude_variance(masked_flow, mask):
    """
    Calculates the variance of optical flow magnitudes within the given mask.
    A high variance suggests non-uniform speeds within the crowd.
    """
    flow_magnitudes = np.linalg.norm(masked_flow[mask > 0], axis=1)
    if len(flow_magnitudes) > 1: # Need at least 2 points for variance calculation
        return np.var(flow_magnitudes)
    return 0.0 # Return 0 if not enough data points

def calculate_directional_coherence(masked_flow, mask):
    """
    Calculates the directional coherence of optical flow within the given mask.
    High coherence (closer to 1.0) means vectors are well-aligned (ordered movement).
    Low coherence (closer to 0.0) means chaotic or random movement.
    """
    valid_flow_vectors = masked_flow[mask > 0] # Extract flow vectors where the mask is active
    
    if len(valid_flow_vectors) == 0:
        return 0.0

    magnitudes = np.linalg.norm(valid_flow_vectors, axis=1)
    non_zero_magnitudes_idx = magnitudes > 1e-6 # Filter out static or near-static pixels
    
    if np.sum(non_zero_magnitudes_idx) == 0:
        return 0.0 # No actual motion to calculate coherence from
        
    unit_vectors = valid_flow_vectors[non_zero_magnitudes_idx] / magnitudes[non_zero_magnitudes_idx, np.newaxis]

    mean_resultant_vector = np.mean(unit_vectors, axis=0) # Average of unit vectors
    coherence = np.linalg.norm(mean_resultant_vector) # Magnitude of the mean resultant vector
    return min(1.0, coherence) # Coherence should be between 0 and 1

def calculate_average_density_in_mask(density_map, mask):
    """
    Calculates the average density within the given mask.
    """
    masked_density = density_map[mask > 0]
    if len(masked_density) > 0:
        return np.mean(masked_density)
    return 0.0

def calculate_enthalpy(masked_flow, density_map):
    """
    Calculates the density-weighted average motion energy (enthalpy proxy) within the crowd.
    Higher enthalpy means more overall kinetic energy/disorder in motion.
    """
    total_weighted_squared_magnitude = 0.0
    total_density_sum = 0.0

    crowd_pixel_indices = np.argwhere(density_map > 0.01) # Filter by significant density

    if len(crowd_pixel_indices) == 0:
        return 0.0

    for y, x in crowd_pixel_indices:
        dx, dy = masked_flow[y, x]
        density_value = density_map[y, x]

        if np.abs(dx) > 0.01 or np.abs(dy) > 0.01: # Filter out near-zero noise motion
            magnitude_squared = dx**2 + dy**2
            total_weighted_squared_magnitude += magnitude_squared * density_value
            total_density_sum += density_value

    if total_density_sum == 0:
        return 0.0

    average_weighted_enthalpy = total_weighted_squared_magnitude / total_density_sum
    return average_weighted_enthalpy

def calculate_flow_divergence(masked_flow, mask):
    """
    Calculates the average divergence of the optical flow field within the crowd mask.
    Positive divergence indicates expansion, negative indicates compression.
    """
    if np.sum(mask) == 0: # No crowd pixels in mask
        return 0.0

    u = masked_flow[:, :, 0] # Horizontal component of flow
    v = masked_flow[:, :, 1] # Vertical component of flow

    # Calculate gradients of u and v using numpy.gradient
    # du/dx (change in horizontal flow over horizontal distance)
    # dv/dy (change in vertical flow over vertical distance)
    du_dx = np.gradient(u, axis=1)
    dv_dy = np.gradient(v, axis=0)

    divergence_map = du_dx + dv_dy

    # Average divergence only over masked crowd pixels
    masked_divergence = divergence_map[mask > 0]
    if len(masked_divergence) > 0:
        return np.mean(masked_divergence)
    return 0.0

def calculate_flow_vorticity(masked_flow, mask):
    """
    Calculates the average vorticity of the optical flow field within the crowd mask.
    Measures rotational motion (swirling).
    """
    if np.sum(mask) == 0: # No crowd pixels in mask
        return 0.0

    u = masked_flow[:, :, 0] # Horizontal component of flow
    v = masked_flow[:, :, 1] # Vertical component of flow

    # Calculate gradients
    # du/dy (change in horizontal flow over vertical distance)
    # dv/dx (change in vertical flow over horizontal distance)
    du_dy = np.gradient(u, axis=0)
    dv_dx = np.gradient(v, axis=1)

    vorticity_map = dv_dx - du_dy # Vorticity for 2D flow

    # Average vorticity only over masked crowd pixels
    masked_vorticity = vorticity_map[mask > 0]
    if len(masked_vorticity) > 0:
        return np.mean(masked_vorticity)
    return 0.0

In [ ]:
def analyze_crowd_motion_in_video(video_path,
                                  output_video_dir="/kaggle/working/crowd_analysis_output/",
                                  density_threshold_for_refinement=0.2,
                                  ema_alpha=0.2,
                                  max_frames_to_process=None,
                                  frames_per_segment=50): # New parameter for segmented output
    """
    Analyzes collective crowd motion in a video and extracts a comprehensive set of per-frame motion metrics.
    Generates an output video visualizing segmentation, optical flow, and density, along with metric overlays.
    Saves the output video in segments for easier viewing and progress tracking.
    """
    os.makedirs(output_video_dir, exist_ok=True)
    video_name_base = os.path.splitext(os.path.basename(video_path))[0]

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return []

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Processing video: {video_path}")
    print(f"Resolution: {frame_width}x{frame_height}, FPS: {fps}, Total Frames: {total_frames}")

    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = None # Initialize video writer to None
    segment_counter = 0 # Track video segments

    # Helper function to open a new VideoWriter for a segment
    def open_new_video_writer(segment_idx):
        segment_output_path = os.path.join(output_video_dir, f"{video_name_base}_segment_{segment_idx:03d}_motion_features_viz.avi")
        print(f"Opening new video segment writer: {segment_output_path}")
        return cv2.VideoWriter(segment_output_path, fourcc, fps, (frame_width * 3, frame_height))

    # Initialize the first video writer
    out = open_new_video_writer(segment_counter)

    ret, prev_frame_bgr = cap.read()
    if not ret:
        print("Failed to read first frame.")
        cap.release()
        if out: out.release()
        return []

    prev_smoothed_density_map = np.zeros((frame_height, frame_width), dtype=np.float32)

    prev_frame_pil_csrnet_input = Image.fromarray(cv2.cvtColor(prev_frame_bgr, cv2.COLOR_BGR2RGB))
    prev_csrnet_input_tensor = manual_image_processor(prev_frame_pil_csrnet_input).unsqueeze(0).to(DEVICE)
    
    with torch.no_grad():
        prev_predicted_density_map_raw = csrnet_model(prev_csrnet_input_tensor)
        prev_current_frame_upsampled_density = torch.nn.functional.interpolate(
            prev_predicted_density_map_raw,
            size=(frame_height, frame_width),
            mode='bilinear',
            align_corners=False).squeeze().cpu().numpy()
            
    prev_smoothed_density_map = prev_current_frame_upsampled_density.copy()

    prev_human_mask_np, _ = get_human_mask_and_density(
        prev_frame_bgr,
        upsampled_density_map=prev_smoothed_density_map,
        density_threshold_for_refinement=density_threshold_for_refinement
    )
    prev_gray = cv2.cvtColor(prev_frame_bgr, cv2.COLOR_BGR2GRAY)

    frame_idx = 1
    start_time = time.time()
    
    all_frame_features = []

    while True:
        # Check if max_frames_to_process limit is reached for overall processing
        if max_frames_to_process is not None and frame_idx > max_frames_to_process:
            print(f"Reached overall max_frames_to_process limit ({max_frames_to_process}). Stopping processing.")
            break

        ret, frame_bgr = cap.read()
        if not ret:
            break

        if frame_idx % 30 == 0 or frame_idx == total_frames - 1:
            print(f"Processing frame {frame_idx}/{total_frames}...")

        current_frame_pil_csrnet_input = Image.fromarray(cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB))
        current_csrnet_input_tensor = manual_image_processor(current_frame_pil_csrnet_input).unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            current_predicted_density_map_raw = csrnet_model(current_csrnet_input_tensor)
            current_frame_upsampled_density = torch.nn.functional.interpolate(
                current_predicted_density_map_raw,
                size=(frame_height, frame_width),
                mode='bilinear',
                align_corners=False).squeeze().cpu().numpy()

        current_smoothed_density_map = (ema_alpha * current_frame_upsampled_density) + \
                                       ((1 - ema_alpha) * prev_smoothed_density_map)

        current_human_mask_np, current_density_map_for_viz = 
            get_human_mask_and_density(
                frame_bgr,
                upsampled_density_map=current_smoothed_density_map,
                density_threshold_for_refinement=density_threshold_for_refinement
            )

        current_human_mask_bool = current_human_mask_np > 0
        prev_human_mask_bool = prev_human_mask_np > 0

        combined_mask_for_flow = (prev_human_mask_bool & current_human_mask_bool).astype(np.uint8) * 255

        current_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, current_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        masked_flow = flow.copy()
        masked_flow[combined_mask_for_flow == 0] = 0

        # --- EXTRACT ALL MOTION & DENSITY METRICS ---
        avg_crowd_speed, dominant_crowd_angle, avg_flow_dx, avg_flow_dy = \
            calculate_density_weighted_flow_metrics(masked_flow, current_smoothed_density_map)
            
        flow_mag_variance = calculate_flow_magnitude_variance(masked_flow, combined_mask_for_flow)
        
        directional_coherence = calculate_directional_coherence(masked_flow, combined_mask_for_flow)
        confusion_index = 1.0 - directional_coherence
        
        total_crowd_count = (current_predicted_density_map_raw).sum().item()
        avg_density_in_mask = calculate_average_density_in_mask(current_smoothed_density_map, current_human_mask_np)

        enthalpy = calculate_enthalpy(masked_flow, current_smoothed_density_map)

        flow_divergence = calculate_flow_divergence(masked_flow, combined_mask_for_flow)
        flow_vorticity = calculate_flow_vorticity(masked_flow, combined_mask_for_flow)

        # --- STORE ALL FEATURES FOR THIS FRAME ---
        frame_features = {
            'video_name': video_name_base,
            'frame_idx': frame_idx,
            'avg_crowd_speed': avg_crowd_speed,
            'dominant_crowd_angle': dominant_crowd_angle,
            'avg_flow_dx': avg_flow_dx,
            'avg_flow_dy': avg_flow_dy,
            'flow_magnitude_variance': flow_mag_variance,
            'directional_coherence': directional_coherence,
            'confusion_index': confusion_index,
            'total_crowd_count': total_crowd_count,
            'avg_density_in_mask': avg_density_in_mask,
            'enthalpy': enthalpy,
            'flow_divergence': flow_divergence,
            'flow_vorticity': flow_vorticity,
        }
        all_frame_features.append(frame_features)
        # --- END FEATURE EXTRACTION ---

        # 4. Visualization
        seg_viz = frame_bgr.copy()
        seg_overlay = np.zeros_like(seg_viz, dtype=np.uint8)
        seg_overlay[current_human_mask_np > 0] = [30, 144, 255]
        seg_viz = cv2.addWeighted(seg_viz, 1, seg_overlay, 0.6, 0)

        masked_flow_hsv_viz = flow_to_hsv(masked_flow)

        density_viz = np.zeros_like(frame_bgr)
        if current_smoothed_density_map.max() > 0:
            normalized_density = cv2.normalize(current_smoothed_density_map, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
            density_viz = cv2.applyColorMap(normalized_density, cv2.COLORMAP_JET)
        else:
            density_viz = np.zeros_like(frame_bgr)

        combined_frame = np.hstack((seg_viz, masked_flow_hsv_viz, density_viz))

        # Text overlays for all metrics
        cv2.putText(combined_frame, f"Video: {video_name_base}", (frame_width * 2 + 10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv2.putText(combined_frame, f"Frame: {frame_idx}/{total_frames}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(combined_frame, f"Smoothed Pred Count: {total_crowd_count:.2f}", (frame_width * 2 + 10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
        
        cv2.putText(combined_frame, f"Avg Speed: {avg_crowd_speed:.2f}", (frame_width * 2 + 10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(combined_frame, f"Dominant Dir: {dominant_crowd_angle:.1f} deg", (frame_width * 2 + 10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(combined_frame, f"Flow Var: {flow_mag_variance:.2f}", (frame_width * 2 + 10, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(combined_frame, f"Dir Coherence: {directional_coherence:.2f}", (frame_width * 2 + 10, 180),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(combined_frame, f"Confusion Index: {confusion_index:.2f}", (frame_width * 2 + 10, 210),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(combined_frame, f"Enthalpy: {enthalpy:.2f}", (frame_width * 2 + 10, 240),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(combined_frame, f"Divergence: {flow_divergence:.2f}", (frame_width * 2 + 10, 270),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(combined_frame, f"Vorticity: {flow_vorticity:.2f}", (frame_width * 2 + 10, 300),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)


        out.write(combined_frame)

        # Check if it's time to close the current segment and open a new one
        if (frame_idx % frames_per_segment == 0 and frame_idx < total_frames):
            print(f"Segment {segment_counter} complete. Closing video writer.")
            out.release()
            segment_counter += 1
            out = open_new_video_writer(segment_counter)
        
        prev_gray = current_gray
        prev_human_mask_np = current_human_mask_np
        prev_smoothed_density_map = current_smoothed_density_map
        prev_frame_bgr = frame_bgr.copy()
        frame_idx += 1

    end_time = time.time()
    print(f"Video processing finished. Total time: {end_time - start_time:.2f} seconds.")
    if out: # Ensure the last video writer is released
        out.release()
        print(f"Final video segment saved in '{output_video_dir}'.")
    cv2.destroyAllWindows()

    return all_frame_features


In [ ]:
# EMOTION_LABELS_MAPPING = {
#     0: 'nothing_emotion',
#     1: 'Angry',
#     2: 'Happy',
#     3: 'Excited',
#     4: 'Scared',
#     5: 'Sad',
#     6: 'Neutral'
# }

# # --- Behavior Label Mapping ---
# BEHAVIOR_LABELS_MAPPING = {
#     0: 'nothing_behavior',
#     1: 'Panic',
#     2: 'Fight',
#     3: 'Congestion',
#     4: 'Obstacle or abnormal object',
#     5: 'Neutral_behavior'
# }

# # --- Function to parse a single MATLAB .m annotation script ---
# def parse_matlab_annotation_script(script_file_path):
#     """
#     Parses a MATLAB .m script file (like dataset_frames_emotion_labeling.m)
#     to extract video labels.
    
#     Args:
#         script_file_path (str): Full path to the .m script file.
        
#     Returns:
#         dict: A dictionary where keys are video IDs (e.g., '001', '002') and
#               values are NumPy arrays of labels for that video.
#               Returns an empty dict if parsing fails or file is not found.
#     """
#     labels_data = {}
#     try:
#         with open(script_file_path, 'r') as f:
#             lines = f.readlines()

#         current_video_id = None
#         current_video_labels_length = 0
#         current_labels_array = None

#         for line in lines:
#             line = line.strip()
#             # Skip comments and empty lines
#             if not line or line.startswith('%'):
#                 continue

#             # Match lines like: video01 = zeros(1,1680);
#             zeros_match = re.match(r'video(\d+) = zeros\(1,(\d+)\);', line)
#             if zeros_match:
#                 current_video_id = f"{int(zeros_match.group(1)):03d}" # Format as '001', '002'
#                 current_video_labels_length = int(zeros_match.group(2))
#                 current_labels_array = np.zeros(current_video_labels_length, dtype=int)
#                 labels_data[current_video_id] = current_labels_array
#                 continue

#             # Match lines like: video01(1,1:1175)=6;
#             assignment_match = re.match(r'video(\d+)\(1,(\d+):(\d+)\)=(\d+);', line)
#             if assignment_match and current_labels_array is not None:
#                 video_id_assigned = f"{int(assignment_match.group(1)):03d}"
#                 start_frame = int(assignment_match.group(2))
#                 end_frame = int(assignment_match.group(3))
#                 label_value = int(assignment_match.group(4))

#                 if video_id_assigned == current_video_id:
#                     # Adjust to 0-based indexing for Python slicing
#                     current_labels_array[start_frame - 1 : end_frame] = label_value
#                 else:
#                     print(f"Warning: Assignment for {video_id_assigned} found, but current_video_id is {current_video_id}. Data might be misaligned.")
#                 continue
            
#             # Match lines like: emotionlabels{1} = video01; or behavelabels{1} = video01;
#             # These lines associate the videoXX array with the global cell array.
#             # We don't need to parse these for direct label extraction, as we're building the videoXX arrays directly.
            
#     except FileNotFoundError:
#         print(f"Error: Script file not found at {script_file_path}")
#     except Exception as e:
#         print(f"Error parsing script {script_file_path}: {e}")
    
#     return labels_data

# # --- Helper function to get labels for a specific video from parsed data ---
# def get_labels_for_video_id(video_id, label_type_key, parsed_labels_data):
#     """
#     Retrieves the label array for a specific video ID and label type
#     from the globally parsed MATLAB script data.
    
#     Args:
#         video_id (str): The 3-digit video ID (e.g., '001').
#         label_type_key (str): Key for the parsed data (e.g., 'emotion' or 'behavior').
#         parsed_labels_data (dict): The global dictionary containing all parsed labels.
        
#     Returns:
#         np.ndarray: A 1D numpy array of numerical labels for that video,
#                     or None if the video ID or label type is not found.
#     """
#     if label_type_key in parsed_labels_data and video_id in parsed_labels_data[label_type_key]:
#         return parsed_labels_data[label_type_key][video_id]
#     print(f"Warning: Labels for video {video_id} ({label_type_key}) not found in parsed data.")
#     return None

In [ ]:
# if __name__ == "__main__":
#     # Define dataset paths
#     dataset_base_path = "/kaggle/input/motion-dataset/" # <--- **UPDATE THIS PATH to your dataset root**
#     video_dir = os.path.join(dataset_base_path, "Motion_Emotion_Dataset")
    
#     # Corrected path based on your provided folder structure
#     annotation_scripts_dir = os.path.join(dataset_base_path, "Motion Emotion Dataset(MED) annotation") 

#     # Paths to your specific .m annotation script files
#     emotion_script_path = os.path.join(annotation_scripts_dir, "dataset_frames_emotion_labeling.m")
#     behavior_script_path = os.path.join(annotation_scripts_dir, "dataset_frames_abnormal_labeling.m") # Assuming abnormal = behavior

#     kaggle_output_base_dir = "/kaggle/working/" # Output directory for generated video and CSV
#     output_analysis_videos_dir = os.path.join(kaggle_output_base_dir, "crowd_analysis_outputs")
#     os.makedirs(output_analysis_videos_dir, exist_ok=True) # Ensure output directory exists

#     print("\n--- Starting Full Feature Extraction and Dataset Creation ---")

#     # --- Step 1: Parse MATLAB .m annotation scripts once ---
#     print(f"Parsing emotion labels from: {emotion_script_path}")
#     parsed_emotion_labels_data = parse_matlab_annotation_script(emotion_script_path)
#     print(f"Parsed {len(parsed_emotion_labels_data)} videos for emotion labels.")

#     print(f"Parsing behavior labels from: {behavior_script_path}")
#     parsed_behavior_labels_data = parse_matlab_annotation_script(behavior_script_path)
#     print(f"Parsed {len(parsed_behavior_labels_data)} videos for behavior labels.")

#     global_parsed_labels = {
#         'emotion': parsed_emotion_labels_data,
#         'behavior': parsed_behavior_labels_data
#     }
    
#     all_videos_features = [] # This list will store features from all processed videos

#     # Discover all video files in the specified directory (assuming .mp4 format)
#     video_files = sorted([f for f in os.listdir(video_dir) if f.endswith('.mp4')])
    
#     # --- OPTIONAL: Limit the number of videos processed for quick testing ---
#     # max_videos_to_process = 2 # Uncomment and set a small number (e.g., 2) for testing
#     # if 'max_videos_to_process' in locals() and max_videos_to_process is not None:
#     #     video_files = video_files[:max_videos_to_process]
    
#     for video_file_name in video_files:
#         full_video_path = os.path.join(video_dir, video_file_name)
#         video_id = os.path.splitext(video_file_name)[0] # Extracts '001' from '001.mp4'

#         # Check if video file exists
#         if not os.path.exists(full_video_path):
#             print(f"Skipping: Video file '{video_file_name}' not found at '{full_video_path}'.")
#             continue

#         print(f"\nProcessing video: {video_file_name}")
        
#         # --- Call the main analysis function to extract per-frame features ---
#         per_frame_features = analyze_crowd_motion_in_video(
#             video_path=full_video_path,
#             output_video_dir=output_analysis_videos_dir,
#             density_threshold_for_refinement=0.2,
#             ema_alpha=0.2,
#             max_frames_to_process=None # Set to None for full video, or e.g., 50 for a quick test
#         )
        
#         # --- Retrieve Labels for the current video from parsed data ---
#         emotion_labels_array = get_labels_for_video_id(video_id, 'emotion', global_parsed_labels)
#         behavior_labels_array = get_labels_for_video_id(video_id, 'behavior', global_parsed_labels)
        
#         # Proceed only if features are extracted AND both label arrays are successfully retrieved
#         if per_frame_features and emotion_labels_array is not None and behavior_labels_array is not None:
#             # Iterate through the extracted features and add the corresponding labels
#             for feature_dict in per_frame_features:
#                 frame_idx_0_based = feature_dict['frame_idx'] - 1 # Convert 1-based frame_idx to 0-based for array indexing
                
#                 # Add Emotion Labels
#                 if frame_idx_0_based < len(emotion_labels_array):
#                     num_emotion_label = int(emotion_labels_array[frame_idx_0_based])
#                     feature_dict['emotion_label_numeric'] = num_emotion_label
#                     feature_dict['emotion_label_str'] = EMOTION_LABELS_MAPPING.get(num_emotion_label, 'Unknown_Emotion')
#                 else:
#                     feature_dict['emotion_label_numeric'] = -1
#                     feature_dict['emotion_label_str'] = 'No_Emotion_Label_Parsed' # Indicate missing label from parsed data

#                 # Add Behavior Labels
#                 if frame_idx_0_based < len(behavior_labels_array):
#                     num_behavior_label = int(behavior_labels_array[frame_idx_0_based])
#                     feature_dict['behavior_label_numeric'] = num_behavior_label
#                     feature_dict['behavior_label_str'] = BEHAVIOR_LABELS_MAPPING.get(num_behavior_label, 'Unknown_Behavior')
#                 else:
#                     feature_dict['behavior_label_numeric'] = -1
#                     feature_dict['behavior_label_str'] = 'No_Behavior_Label_Parsed' # Indicate missing label from parsed data

#             all_videos_features.extend(per_frame_features) # Add features from current video to master list
#             print(f"Finished processing {video_file_name}. Extracted {len(per_frame_features)} frames.")
#         else:
#             print(f"Skipping {video_file_name} due to missing extracted features or labels not found in parsed data.")

#     print("\n--- Feature Extraction and Label Integration Complete. Building DataFrame. ---")

#     if not all_videos_features:
#         print("No features extracted from any video. Exiting the ML pipeline.")
#     else:
#         # Create a Pandas DataFrame from all collected per-frame features
#         df_full_dataset = pd.DataFrame(all_videos_features)
        
#         # --- Data Cleaning/Filtering for ML ---
#         # Filter out frames with no valid emotion label for the emotion prediction stage.
#         df_emotion_data = df_full_dataset[df_full_dataset['emotion_label_str'] != 'No_Emotion_Label_Parsed'].copy()
#         df_emotion_data = df_emotion_data[df_emotion_data['emotion_label_str'] != 'nothing_emotion'].copy()
        
#         # Filter out frames with no valid behavior label for the behavior prediction stage.
#         df_behavior_data = df_full_dataset[df_full_dataset['behavior_label_str'] != 'No_Behavior_Label_Parsed'].copy()
#         df_behavior_data = df_behavior_data[df_behavior_data['behavior_label_str'] != 'nothing_behavior'].copy()

#         # Define the common motion feature columns used as input to both models
#         motion_feature_columns = [
#             'avg_crowd_speed', 'dominant_crowd_angle', 'avg_flow_dx', 'avg_flow_dy',
#             'flow_magnitude_variance', 'directional_coherence', 'confusion_index',
#             'total_crowd_count', 'avg_density_in_mask', 'enthalpy',
#             'flow_divergence', 'flow_vorticity'
#         ]
        
#         # --- Stage 1: Train Emotion Classifier (Motion Features -> Emotion Labels) ---
#         print("\n--- Stage 1: Training Emotion Classifier (Motion Features -> Emotion Labels) ---")
#         if not df_emotion_data.empty:
#             X_emotion = df_emotion_data[motion_feature_columns]
#             y_emotion = df_emotion_data['emotion_label_str']

#             emotion_label_encoder = LabelEncoder()
#             y_emotion_encoded = emotion_label_encoder.fit_transform(y_emotion)
            
#             # Split emotion data
#             X_emotion_train, X_emotion_temp, y_emotion_train, y_emotion_temp = train_test_split(
#                 X_emotion, y_emotion_encoded, test_size=0.3, random_state=42, stratify=y_emotion_encoded)
#             X_emotion_val, X_emotion_test, y_emotion_val, y_emotion_test = train_test_split(
#                 X_emotion_temp, y_emotion_temp, test_size=0.5, random_state=42, stratify=y_emotion_temp)

#             # Scale emotion features
#             emotion_scaler = StandardScaler()
#             X_emotion_train_scaled = emotion_scaler.fit_transform(X_emotion_train)
#             X_emotion_val_scaled = emotion_scaler.transform(X_emotion_val)
#             X_emotion_test_scaled = emotion_scaler.transform(X_emotion_test)

#             # Train Emotion Model
#             emotion_model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
#             emotion_model.fit(X_emotion_train_scaled, y_emotion_train)
#             print("Emotion Classifier Training Complete.")

#             # Evaluate Emotion Model
#             y_emotion_val_pred = emotion_model.predict(X_emotion_val_scaled)
#             print("\n--- Emotion Classifier Validation Set Performance ---")
#             print(f"Accuracy: {accuracy_score(y_emotion_val, y_emotion_val_pred):.4f}")
#             print("Classification Report:\n", classification_report(y_emotion_val, y_emotion_val_pred, target_names=emotion_label_encoder.classes_))
#             print("Confusion Matrix:\n", confusion_matrix(y_emotion_val, y_emotion_val_pred))
            
#             y_emotion_test_pred = emotion_model.predict(X_emotion_test_scaled)
#             print("\n--- Emotion Classifier Test Set Performance ---")
#             print(f"Accuracy: {accuracy_score(y_emotion_test, y_emotion_test_pred):.4f}")
#             print("Classification Report:\n", classification_report(y_emotion_test, y_emotion_test_pred, target_names=emotion_label_encoder.classes_))
#             print("Confusion Matrix:\n", confusion_matrix(y_emotion_test, y_emotion_test_pred))

#         else:
#             print("No data available for emotion prediction. Skipping Stage 1 (Emotion Classifier).")
#             emotion_model = None # Ensure emotion_model is None if not trained

#         # --- Stage 2: Train Behavior Classifier (Motion Features + Predicted Emotion Probabilities -> Behavior Labels) ---
#         print("\n--- Stage 2: Training Behavior Classifier (Motion Features + Inferred Emotion -> Behavior Labels) ---")
#         if not df_behavior_data.empty and emotion_model is not None:
#             # We need to ensure X_behavior_motion_raw is aligned with df_behavior_data's indices
#             # and that it's transformed using the *same scaler* as emotion model was trained on.
#             # Make sure we use the original (unscaled) motion features from df_behavior_data
#             # when predicting emotion probabilities, then scale the *combined* features for behavior model.

#             # Step 1: Predict emotion probabilities using the trained emotion model
#             # and the *original motion features* for the behavior dataset's frames.
#             # Important: Use the same scaler that was fitted on the emotion training data.
#             X_behavior_motion_for_emotion_pred = emotion_scaler.transform(df_behavior_data[motion_feature_columns])
#             predicted_emotion_probabilities = emotion_model.predict_proba(X_behavior_motion_for_emotion_pred)

#             # Convert predicted probabilities to a DataFrame and add to the main feature set
#             predicted_emotion_df = pd.DataFrame(
#                 predicted_emotion_probabilities,
#                 columns=[f'pred_emotion_prob_{cls}' for cls in emotion_label_encoder.classes_],
#                 index=df_behavior_data.index # Crucial: preserve original index for alignment
#             )
            
#             # Combine original motion features with predicted emotion probabilities
#             X_behavior_combined = pd.concat([df_behavior_data[motion_feature_columns], predicted_emotion_df], axis=1)
#             y_behavior = df_behavior_data['behavior_label_str'] # Target labels for behavior

#             behavior_label_encoder = LabelEncoder()
#             y_behavior_encoded = behavior_label_encoder.fit_transform(y_behavior)

#             # Split behavior data using indices to ensure consistent splits across X and y
#             train_idx, temp_idx, _, _ = train_test_split(
#                 X_behavior_combined.index, y_behavior_encoded, test_size=0.3, random_state=42, stratify=y_behavior_encoded)
#             val_idx, test_idx, _, _ = train_test_split(
#                 temp_idx, y_behavior_encoded[temp_idx], test_size=0.5, random_state=42, stratify=y_behavior_encoded[temp_idx])

#             # Select data using the generated indices
#             X_behavior_train_combined = X_behavior_combined.loc[train_idx]
#             y_behavior_train = y_behavior_encoded[train_idx]
            
#             X_behavior_val_combined = X_behavior_combined.loc[val_idx]
#             y_behavior_val = y_behavior_encoded[val_idx]

#             X_behavior_test_combined = X_behavior_combined.loc[test_idx]
#             y_behavior_test = y_behavior_encoded[test_idx]
            
#             # Scale the combined features for the behavior model. A new scaler is used.
#             behavior_scaler = StandardScaler()
#             X_behavior_train_scaled = behavior_scaler.fit_transform(X_behavior_train_combined)
#             X_behavior_val_scaled = behavior_scaler.transform(X_behavior_val_combined)
#             X_behavior_test_scaled = behavior_scaler.transform(X_behavior_test_combined)

#             # Train Behavior Model
#             behavior_model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
#             behavior_model.fit(X_behavior_train_scaled, y_behavior_train)
#             print("Behavior Classifier Training Complete.")

#             # Evaluate Behavior Model
#             y_behavior_val_pred = behavior_model.predict(X_behavior_val_scaled)
#             print("\n--- Behavior Classifier Validation Set Performance ---")
#             print(f"Accuracy: {accuracy_score(y_behavior_val, y_behavior_val_pred):.4f}")
#             print("Classification Report:\n", classification_report(y_behavior_val, y_behavior_val_pred, target_names=behavior_label_encoder.classes_))
#             print("Confusion Matrix:\n", confusion_matrix(y_behavior_val, y_behavior_val_pred))
            
#             y_behavior_test_pred = behavior_model.predict(X_behavior_test_scaled)
#             print("\n--- Behavior Classifier Test Set Performance ---")
#             print(f"Accuracy: {accuracy_score(y_behavior_test, y_behavior_test_pred):.4f}")
#             print("Classification Report:\n", classification_report(y_behavior_test, y_behavior_test_pred, target_names=behavior_label_encoder.classes_))
#             print("Confusion Matrix:\n", confusion_matrix(y_behavior_test, y_behavior_test_pred))

#         else:
#             print("No data available for behavior prediction or emotion model not trained. Skipping Stage 2 (Behavior Classifier).")

#     print("\n--- Overall Crowd Behavior Analysis Pipeline Complete ---")


In [ ]:
if __name__ == "__main__":
    # Define dataset paths (if you need to iterate through multiple videos later)
    # dataset_base_path = "/kaggle/input/motion-dataset/"
    # video_dir = os.path.join(dataset_base_path, "Motion_Emotion_Dataset")
    
    # Output directory for generated videos and collected feature data
    kaggle_output_base_dir = "/kaggle/working/"
    output_analysis_videos_dir = os.path.join(kaggle_output_base_dir, "crowd_motion_analysis_results")
    os.makedirs(output_analysis_videos_dir, exist_ok=True)

    print("\n--- Starting Crowd Motion Feature Extraction ---")

    all_extracted_features = [] # List to collect features from all videos

    # Hardcoded video path as per your request for a single video
    video_path_to_process = '/kaggle/input/new-dataset/1.mp4'
    
        # Call the main analysis function with segmented output enabled
    per_frame_features = analyze_crowd_motion_in_video(
        video_path=video_path_to_process,
        output_video_dir=output_analysis_videos_dir,
        density_threshold_for_refinement=0.2,
        ema_alpha=0.2,
        max_frames_to_process=None, # Process full video
        frames_per_segment=50 # Save video in 50-frame segments
        )
        
    if per_frame_features:
        all_extracted_features.extend(per_frame_features)
        print(f"Extracted {len(per_frame_features)} frames of motion features for {video_file_name_base}.")
    else:
        print(f"No motion features extracted for {video_file_name_base}. It might be corrupted or empty.")

    print("\n--- All Video Processing Complete ---")
    print(f"Total frames with features extracted across all videos: {len(all_extracted_features)}")

    # If you later want to convert this list to a DataFrame or save it:
    # import pandas as pd
    # df_all_features = pd.DataFrame(all_extracted_features)
    # csv_output_path = os.path.join(kaggle_output_base_dir, "all_crowd_motion_features.csv")
    # df_all_features.to_csv(csv_output_path, index=False)
    # print(f"All extracted features saved to: {csv_output_path}")

    print("\nMotion feature extraction pipeline finished. Segmented output videos are saved in the 'crowd_motion_analysis_results' directory.")
